In [1]:
import numpy as np
import random
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from keras.utils import to_categorical
import math

random.seed(42)
np.random.seed(42)

In [2]:
# Load and prepare MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255
train_images = train_images.reshape((train_images.shape[0], 784))
test_images = test_images.reshape((test_images.shape[0], 784))
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.99, random_state=42
)

In [3]:
# Define ReLU, SSE and Logistic functions
def logistic(x):
    return 1 / (1 + math.exp(-x))

def relu(x):
    return np.maximum(0, x)

def sum_of_squared_relu(x):
    return np.sum(relu(x)**2)

def derivative_sum_of_squared_relu(x):
    return 2 * relu(x) * (x > 0)

In [4]:
def encode_lables(input_data, class_index, num_classes=10):
    modified_input = np.copy(input_data)
    class_vector = np.zeros(num_classes)
    class_vector[class_index] = 1
    modified_input[:10] = class_vector  # Replace first 10 pixels
    return modified_input

In [5]:
# Initialize network
def initialize_network(layer_sizes):
    return [
        {
            'weights': np.random.rand(layer_sizes[i], layer_sizes[i+1]),
            'biases': np.random.rand(layer_sizes[i+1])
        }
        for i in range(len(layer_sizes) - 1)
    ]

initialize_network([2,1,2])

[{'weights': array([[0.37454012],
         [0.95071431]]),
  'biases': array([0.73199394])},
 {'weights': array([[0.59865848, 0.15601864]]),
  'biases': array([0.15599452, 0.05808361])}]

In [6]:
import matplotlib.pyplot as plt

def display_image(image_vector):
    image_matrix = image_vector.reshape(28, 28)  # Reshape from 784 to 28x28
    plt.imshow(image_matrix, cmap='gray')
    plt.colorbar()
    plt.show()

# display_image(encode_lables(test_images[0], np.argmax(train_labels[0])))

In [7]:
def l2norm(vector):
    return vector / np.linalg.norm(vector)

In [8]:
def predict(network, val_image, num_classes = 10):
    predictions = []
    # display_image(val_image)
    for image_class in range(num_classes):
        # print(image_class)
        prediction = []
        inputs = encode_lables(val_image, image_class)
        for layer in network:
            inputs = np.dot(inputs, layer['weights']) + layer['biases']
            prediction.append(logistic(sum_of_squared_relu(inputs)))
        # print(image_class, ': ', np.mean(prediction), ': ', prediction)
        predictions.append(np.mean(prediction))
    # print('Predicted class: ', np.argmax(predictions))
    return np.argmax(predictions)

def accuracy(network, images, labels):
    predictions = []
    for i in range(len(images)):
        predicted_class = predict(network, images[i:i+1][0])
        real_class = np.argmax(labels[i:i+1], axis=1)[0]
        predictions.append(predicted_class == real_class)
        # print(predicted_class)
        # print(real_class)
        # print(predictions)
    return np.mean(predictions)

In [9]:
# Forward propagate with detailed logging
def train_network(network, train_images, train_labels, val_images, val_labels, epochs, learning_rate):
    for epoch in range(epochs):
        # Training loop
        for i in range(len(train_images)):
            for l, layer in enumerate(network):
                # positive data
                if l == 0:
                    image_class = np.argmax(train_labels[i:i+1], axis=1)[0]
                    pos_inputs = encode_lables(train_images[i:i+1][0], image_class)
                goodness = sum_of_squared_relu(np.dot(pos_inputs, layer['weights']) + layer['biases'])
                grad = derivative_sum_of_squared_relu(np.dot(pos_inputs, layer['weights']) + layer['biases'])
                layer['weights'] += learning_rate * grad
                layer['biases'] += learning_rate * grad
                # negative data
                if l == 0:
                    not_image_class = random.choice([i for i in range(10) if i != image_class])
                    neg_inputs = encode_lables(train_images[i:i+1][0], not_image_class)
                goodness = sum_of_squared_relu(np.dot(neg_inputs, layer['weights']) + layer['biases'])
                grad = derivative_sum_of_squared_relu(np.dot(neg_inputs, layer['weights']) + layer['biases'])
                layer['weights'] -= learning_rate * grad
                layer['biases'] -= learning_rate * grad

                pos_inputs = l2norm(np.dot(pos_inputs, layer['weights']) + layer['biases'])
                neg_inputs = l2norm(np.dot(neg_inputs, layer['weights']) + layer['biases'])

        print(f"Epoch {epoch+1}/{epochs}")
        train_accuracy = accuracy(network, train_images, train_labels)
        print(f"Training Accuracy: {train_accuracy:.4f}")
        val_accuracy = accuracy(network, val_images[:60], val_labels[:60])
        print(f"Test Accuracy: {val_accuracy:.4f}")
    # predict(network, val_images[0])

learning_rate = 0.002
epochs = 20
network = initialize_network([784, 128, 64, 10])
train_network(network, train_images, train_labels, val_images, val_labels, epochs, learning_rate)
final_accuracy = accuracy(network, test_images[:60], test_labels[:60])
print("Final Accuracy:", final_accuracy)

Epoch 1/20
Training Accuracy: 0.1617
Test Accuracy: 0.1667
Epoch 2/20
Training Accuracy: 0.1733
Test Accuracy: 0.1667
Epoch 3/20
Training Accuracy: 0.1900
Test Accuracy: 0.1833
Epoch 4/20
Training Accuracy: 0.1850
Test Accuracy: 0.2000
Epoch 5/20
Training Accuracy: 0.1817
Test Accuracy: 0.1833
Epoch 6/20
Training Accuracy: 0.1817
Test Accuracy: 0.1667
Epoch 7/20
Training Accuracy: 0.1950
Test Accuracy: 0.1833
Epoch 8/20
Training Accuracy: 0.1950
Test Accuracy: 0.2000
Epoch 9/20
Training Accuracy: 0.2100
Test Accuracy: 0.1833
Epoch 10/20
Training Accuracy: 0.1933
Test Accuracy: 0.1833
Epoch 11/20
Training Accuracy: 0.2067
Test Accuracy: 0.1833
Epoch 12/20
Training Accuracy: 0.1983
Test Accuracy: 0.1833
Epoch 13/20
Training Accuracy: 0.2100
Test Accuracy: 0.2000
Epoch 14/20
Training Accuracy: 0.2000
Test Accuracy: 0.2000
Epoch 15/20
Training Accuracy: 0.1900
Test Accuracy: 0.2000
Epoch 16/20
Training Accuracy: 0.1983
Test Accuracy: 0.2167
Epoch 17/20
Training Accuracy: 0.1983
Test Accura